In [7]:
import collections

In [1]:
class Int:
    
    def __init__(self, min_value=None, max_value=None):
        self.min_value=min_value
        self.max_value=max_value
        
    def __set_name__(self, owner_class, name):
        self.name=name
    
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise ValueError(f"{self.name} must be an int.")
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f"{self.name} must be at least {self.min_value}")
            
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"{self.name} cannot be greater than {self.max_value}")
        instance.__dict__[self.name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.name)
            

In [2]:
class Point2D:
    x = Int(min_value=0, max_value=800)
    y = Int(min_value=0, max_value=600)
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Point2D(x={self.x}, y={self.y})"
    
    def __str__(self):
        return f"({self.x}, {self.y})"

In [3]:
p = Point2D(0, 10)
str(p)

'(0, 10)'

In [4]:
p

Point2D(x=0, y=10)

In [6]:
p = Point2D(0, 800)

ValueError: y cannot be greate than 600

#### sequence

In [8]:
isinstance([1,2,3], collections.abc.Sequence)

True

In [9]:
isinstance((1,2,3), collections.abc.Sequence)

True

set is not a sequence

In [10]:
isinstance({1,2,3}, collections.abc.Sequence)

False

In [11]:
isinstance([1,2,3], collections.abc.MutableSequence)

True

In [12]:
isinstance((1,2,3), collections.abc.MutableSequence)

False

In [14]:
class Point2DSequence:
    
    def __init__(self, min_length=None, max_length=None):
        self.min_length=min_length
        self.max_length=max_length
        
    def __set_name__(self, owner_class, name):
        self.name=name
    
    def __set__(self, instance, value):
        if not isinstance(value, collections.abc.Sequence):
            raise ValueError(f"{self.name} must be a sequence type.")
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f"{self.name} must contain at least {self.min_length} elements.")
            
        if self.max_length is not None and len(value) > self.max_length:
            raise ValueError(f"{self.name} cannot contain more than {self.max_length} elements.")
        
        for index, item in enumerate(value):
            if not isinstance(item, Point2D):
                raise ValueError(f"Item at index {index} is not a Point2D instance.")
        instance.__dict__[self.name] = list(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            if self.name not in instance.__dict__:
                instance.__dict__[self.name] = []
            return instance.__dict__[self.name]
            

In [15]:
class Polygon:
    vertices = Point2DSequence(min_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices

In [16]:
try:
    p = Polygon()
except ValueError as ex:
    print(ex)

vertices must contain at least 3 elements.


In [17]:
try:
    p = Polygon(Point2D(-100, 0), Point2D(0, 1), Point2D(1, 0),)
except ValueError as ex:
    print(ex)

x must be at least 0


In [18]:
p = Polygon(Point2D(0, 0), Point2D(0, 1), Point2D(1, 0),)

In [19]:
p.vertices

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0)]

In [22]:
class Polygon:
    vertices = Point2DSequence(min_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices
        
    def append(self, pt):
        if not isinstance(pt, Point2D):
            raise ValueError("Can only append Point2D objects.")
            
        max_length = type(self).vertices.max_length
        if max_length is not None and len(self.vertices) >= max_length:
            raise ValueError(f"Vertices length is at max {max_length}.")
        self.vertices.append(pt)
            
            

In [23]:
p = Polygon(Point2D(0, 0), Point2D(0, 1), Point2D(1, 0),)

In [24]:
p.vertices

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0)]

In [25]:
p.append(Point2D(34, 59))

In [29]:
class Triangle:
    vertices = Point2DSequence(min_length=3, max_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices
        
    def append(self, pt):
        if not isinstance(pt, Point2D):
            raise ValueError("Can only append Point2D objects.")
            
        max_length = type(self).vertices.max_length
        if max_length is not None and len(self.vertices) >= max_length:
            raise ValueError(f"Vertices length is at max ({max_length}).")
        self.vertices.append(pt)
            

In [30]:
p = Triangle(Point2D(0, 0), Point2D(0, 1), Point2D(1, 0),)

In [31]:
p.append(Point2D(34, 59))

ValueError: Vertices length is at max (3).

#### use inheritance

In [32]:
class Polygon:
    vertices = Point2DSequence(min_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices
        
    def append(self, pt):
        if not isinstance(pt, Point2D):
            raise ValueError("Can only append Point2D objects.")
            
        max_length = type(self).vertices.max_length
        if max_length is not None and len(self.vertices) >= max_length:
            raise ValueError(f"Vertices length is at max ({max_length}).")
        self.vertices.append(pt)
            

In [33]:
class Triangle(Polygon):
    vertices = Point2DSequence(min_length=3, max_length=3)

In [34]:
class Rectangle(Polygon):
    vertices = Point2DSequence(min_length=4, max_length=4)

In [35]:
t = Triangle(Point2D(0, 0), Point2D(0, 1), Point2D(1, 0),Point2D(2, 2))

ValueError: vertices cannot contain more than 3 elements.

#### we can make Polygon class into a sequence type itself

In [44]:
class Polygon:
    vertices = Point2DSequence(min_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices
        
    def append(self, pt):
        if not isinstance(pt, Point2D):
            raise ValueError("Can only append Point2D objects.")
            
        max_length = type(self).vertices.max_length
        if max_length is not None and len(self.vertices) >= max_length:
            raise ValueError(f"Vertices length is at max ({max_length}).")
        self.vertices.append(pt)
        
    def __len__(self):
        return len(self.vertices)
    
    def __getitem__(self, idx):
        return self.vertices[idx]
    
    def __iadd__(self, pt):
        self.append(pt)
        return self
    
    def __contains__(self, pt):
        return pt in self.vertices
            

In [45]:
p = Polygon(Point2D(0, 0), Point2D(0, 1), Point2D(1, 0),)

In [46]:
p[1]

Point2D(x=0, y=1)

In [47]:
p[1:3]

[Point2D(x=0, y=1), Point2D(x=1, y=0)]

In [51]:
p.vertices

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0)]

In [49]:
p[0]

Point2D(x=0, y=0)

In [50]:
list(p)

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0)]

In [52]:
p += Point2D(11,12)

In [54]:
list(p)

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0), Point2D(x=11, y=12)]

In [55]:
Point2D(11,12) in p # we need to implement __eq__ method

False

In [56]:
class Point2D:
    x = Int(min_value=0, max_value=800)
    y = Int(min_value=0, max_value=600)
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Point2D(x={self.x}, y={self.y})"
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __eq__(self, other):
        return isinstance(other, Point2D) and self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))

class Polygon:
    vertices = Point2DSequence(min_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices
        
    def append(self, pt):
        if not isinstance(pt, Point2D):
            raise ValueError("Can only append Point2D objects.")
            
        max_length = type(self).vertices.max_length
        if max_length is not None and len(self.vertices) >= max_length:
            raise ValueError(f"Vertices length is at max ({max_length}).")
        self.vertices.append(pt)
        
    def __len__(self):
        return len(self.vertices)
    
    def __getitem__(self, idx):
        return self.vertices[idx]
    
    def __iadd__(self, pt):
        self.append(pt)
        return self
    
    def __contains__(self, pt):
        return pt in self.vertices
            

In [57]:
p = Polygon(Point2D(0, 0), Point2D(0, 1), Point2D(1, 0),)

In [59]:
Point2D(0, 0) in p

True

In [60]:
Point2D(11,12) in p 

False